In [2]:
####Only SBIR now
#### abstract can have "\n"
###

from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [156]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': ' ', '\t': ' ', '    ': '', '&emsp': ' ',
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = str.strip('\n')
    str = str.strip(' ')

    return str

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./STTRResult/00_STTR_phase2.csv', index=False, encoding="utf-8")
    
def ReadFiles(Directory_path):  # Read all the html files
    path = Directory_path  # The directory
    file_names = os.listdir(path)  # 得到文件夹下的所有文件名称
    files_position = []
    for file_name in file_names:
        position = path + "/" + file_name
        # print(position)
        files_position.append(position)

    print("Total number of HTML files: ", len(files_position))
    return files_position


def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

In [159]:
def SingleHTMLProcess(path):
    dic = {"Proposal Number": "", "Phase 1 Contract #": "", "Subtopic Title": "", "Proposal Title": "",
           "Small Business Concern_Firm": "", "Small Business Concern_Address": "", "Small Business Concern_Phone": "",
           "Principal Investigator_Name": "", "Principal Investigator_E-mail": "", "Principal Investigator_Address": "",
           "Principal Investigator_Phone": "",
           "Business Official_Name": "", "Business Official_E-mail": "", "Business Official_Address": "",
           "Business Official_Phone": "",
           "TRL_Begin": "", "TRL_End": "", "Technical Abstract": "", "Potential NASA applications": "",
           "Potential non-NASA applications": "","Technology Taxonomy Mapping":"", 'Research Topic':"", 'Research Institution Name':"", 'Research Institution Address':"",
          'Research Institution Phone':""}
    htmlfile = open(path, 'r', encoding='utf-8')
    html=htmlfile  
    bs = BeautifulSoup(html, "lxml")  # 缩进格式
    print(path)
    dic['Technical Abstract'] = filterHTMLstr(getTechAbstract(bs))
    dic['Potential NASA applications'] = filterHTMLstr(getNonNASAApplication(bs))
    dic['Proposal Number'] = filterHTMLstr(getProposalNumber(bs))
    dic['Research Topic'] = filterHTMLstr(getResearchTopic(bs))
    dic['Proposal Title'] = filterHTMLstr(getProposalTitle(bs))
    dic['Phase 1 Contract #'] = filterHTMLstr(getPhase1ContractNum(bs))
    dic['Small Business Concern_Firm'] = filterHTMLstr(getSMBAndRIInfo(bs)[0])
    dic['Small Business Concern_Address'] = filterHTMLstr(getSMBAndRIInfo(bs)[2])
    dic['Small Business Concern_Phone'] = filterHTMLstr(getSMBAndRIInfo(bs)[4])
    dic['Research Institution Name'] = filterHTMLstr(getSMBAndRIInfo(bs)[1])
    dic['Research Institution Address'] = filterHTMLstr(getSMBAndRIInfo(bs)[3])
    dic['Research Institution Phone'] = filterHTMLstr(getSMBAndRIInfo(bs)[5])
    dic['Principal Investigator_Name'] = filterHTMLstr(getPIName(bs))
    return dic

In [160]:
Directory_path = "../Datasets/00/sttr/phase2"
files_position = ReadFiles(Directory_path)
totaldata = MultipleFileProcess(files_position)
to_CSV(totaldata)

Total number of HTML files:  9
../Datasets/00/sttr/phase2/STTR-00-2-01-000056.html
Somnath Deb
../Datasets/00/sttr/phase2/STTR-00-2-02-000122.html
Dr. Robert Stirling
../Datasets/00/sttr/phase2/STTR-00-2-03-000038.html
John Lowrance
../Datasets/00/sttr/phase2/STTR-00-2-05-000079.html
Kristie Cooper
../Datasets/00/sttr/phase2/STTR-00-2-01-000118.html
Dr. James C. Neidhoefer
../Datasets/00/sttr/phase2/STTR-00-2-03-000070.html
Dr. Andrea E. Hoyt
../Datasets/00/sttr/phase2/STTR-00-2-04-000141.html
Dr. Andrew V. Pakhomov
../Datasets/00/sttr/phase2/STTR-00-2-02-000063.html
Dr. Ravi Prasanth
../Datasets/00/sttr/phase2/STTR-00-2-03-000011.html
Nahum Gat


### Test

In [5]:
path = "../Datasets/00/sttr/phase2/STTR-00-2-01-000056.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式

In [26]:
### Proposal Number
def getProposalNumber(bs):
    proposalMine = bs.find_all("tr")[0].get_text()
    regProposalNum = re.compile(r'\n *([0-9]*)', re.S)
    proposalNumber = re.findall(regProposalNum, proposalMine)[0]
    return proposalNumber
print(getProposalNumber(bs))

000056


In [40]:
def getPhase1ContractNum(bs):
    proposalMine = bs.find_all("tr")[1].get_text()
    regProposalNum = re.compile(r'#:\n *(.*?) *\n', re.S)
    proposalNumber = re.findall(regProposalNum, proposalMine)[0]
    return proposalNumber
print(getPhase1ContractNum(bs))

NAS2 01018


In [67]:
def getResearchTopic(bs):
    proposalMine = bs.find_all("tr")[2].get_text()
    regProposalNum = re.compile(r'\nRESEARCH\xa0TOPIC: (.*) *', re.S)
    proposalNumber = re.findall(regProposalNum, proposalMine)[0]
    return proposalNumber
print(getResearchTopic(bs))

01-Information Technology  


In [69]:
def getProposalTitle(bs):
    proposalMine = bs.find_all("tr")[3].get_text()
    regProposalNum = re.compile(r'\nPROPOSAL\xa0TITLE: (.*) *', re.S)
    proposalNumber = re.findall(regProposalNum, proposalMine)[0]
    return proposalNumber
print(getProposalTitle(bs))

Real-time Onboard and Remote Vehicle Health Management 


In [135]:
def getSMBAndRIInfo(bs):
    nameMine = bs.find_all("tr")[5].get_text()
    regSMBName = re.compile(r'\nNAME: (.*?) *\nNAME', re.S)
    regRIName = re.compile(r'\nNAME: .*? *\nNAME: *(.*?) *\n', re.S)
    addressMine = bs.find_all("tr")[6].get_text()
    regSMBAddress1 = re.compile('\nADDRESS: (.*?)ADDRESS', re.S)
    regRIAddress1 = re.compile('.*?ADDRESS:.*?ADDRESS: (.*?)\n', re.S)
    addressMine2 = bs.find_all("tr")[7].get_text()
    regSMBAddress2 = re.compile('\nCITY: (.*?)CITY', re.S)
    regRIAddress2 = re.compile('.*?CITY:.*?CITY: (.*?)\n', re.S)
    addressMine3 = bs.find_all("tr")[8].get_text()
    regAddress3 = re.compile('\nSTATE/ZIP:(.*?) \xa0 (.*?) \n', re.S)
    phoneMine = bs.find_all("tr")[9].get_text()
    regPhone = re.compile('\nPHONE: (.*?) *\n', re.S)
    
    smbName = re.findall(regSMBName, nameMine)[0]
    riName = re.findall(regRIName, nameMine)[0]
    address3 = re.findall(regAddress3, addressMine3)
    smbAddress = re.findall(regSMBAddress1, addressMine)[0].strip() + ", " + re.findall(regSMBAddress2, addressMine2)[0].strip() + ", " + address3[0][0].strip() + ", " + address3[0][1].strip()
    riAddress = re.findall(regRIAddress1, addressMine)[0].strip() + ", " + re.findall(regRIAddress2, addressMine2)[0].strip() + ", " + address3[1][0].strip() + ", " + address3[1][1].strip()
    phone = re.findall(regPhone, phoneMine)
    smbPhone = phone[0]
    riPhone = phone[1]
    return [smbName, riName, smbAddress, riAddress, smbPhone, riPhone]
print(getSMBAndRIInfo(bs))

['Qualtech Systems, Inc.', 'University of Connecticut', '100 Great Meadow Road, Suite 501, Wethersfield, CT, 06109 - 2355', 'U-157, 260 Glenbrook Road,, Storrs, CT, 06269 - 2157', '(860 )\xa0257 - 8014', '(860 )\xa0486 - 2195']


In [139]:
def getPIName(bs):
    field = bs.find_all("tr")[10].get_text()
    reg = re.compile('\nPRINCIPAL INVESTIGATOR/PROJECT MANAGER:\n \xa0(.*?)\n', re.S)
    name = re.findall(reg, field)[0]
    return name

Somnath Deb
Somnath Deb


In [153]:
### Technical Abstract and Potential Commercial Application
def getTechAbstract(bs):
    technicalAbstract = bs.find_all("p")[2].get_text()
    return technicalAbstract

def getNonNASAApplication(bs):
    nonNASAApplication = bs.find_all("p")[3].get_text()
    return nonNASAApplication